&emsp;&emsp; 1. Знайти кількість злочинів, що сталися у грудні 2012 року в інтервалі між першою
годиною ночі та п&#39;ятою ранку (CrimesData)

In [220]:
import pandas as pd
import numpy as np

crimes_data = pd.read_excel( 
    '../Lessons/Lesson 5/Files/CrimesData.xlsx',
    sheet_name='2012',
    usecols='A:B, K')

# приведемо колонку CrimeDate до часового формату
crimes_data['CrimeDate'] = pd.to_datetime(
    crimes_data['CrimeDate'],
    format='%m/%d/%Y'
)

# приведемо колонку час до правильного формату
def fix_time(cell):
    """1937 -> 19:37:00"""
    if len(cell) == 4 and ':' not in cell:
        return f'{cell[:2]}:{cell[2:]}:00'
    else:
        return cell
  
crimes_data['CrimeTime'] = crimes_data['CrimeTime'].apply(fix_time)  # внесемо правки до оригінальної колонки

# приведемо колонку CrimeTime до часового формату
crimes_data['CrimeTime'] = pd.to_datetime(
    crimes_data['CrimeTime']
)

# застосовуємо фільтри для місяця і години
count = crimes_data[(crimes_data['CrimeDate'].dt.month == 12) & (crimes_data['CrimeTime'].dt.hour >= 1) & (crimes_data['CrimeTime'].dt.hour <= 5)].count()
print('Кількість злочинів', np.sum(count['Total Incidents']))

Кількість злочинів 44


&emsp;&emsp; 2. Знайти середню, мінімальну і максимальну вартість квитка в залежності від місця посадки і класу каюти (`Титанік`)

In [221]:
titanic_data = pd.read_csv(
    '../Lessons/Lesson 5/Files/TitanicDataset.csv',
    sep=',',
    header=0)

aggr_data = titanic_data.groupby(by=['Embarked', 'Pclass']).agg(
    mean_fare=('Fare', np.mean),
    min_fare=('Fare', np.min),
    max_fare=('Fare', np.max)
)
aggr_data


mean_fare  min_fare  max_fare
Embarked Pclass                                
C        1       104.718529   26.5500  512.3292
         2        25.358335   12.0000   41.5792
         3        11.214083    4.0125   22.3583
Q        1        90.000000   90.0000   90.0000
         2        12.350000   12.3500   12.3500
         3        11.183393    6.7500   29.1250
S        1        70.364862    0.0000  263.0000
         2        20.327439    0.0000   73.5000
         3        14.644083    0.0000   69.5500


&emsp;&emsp; 3. Найти загальну суму продажу по місяцях 2019-го року; вивести в форматі місяця - сума в порядку збільшення суми (`Sales`)

In [222]:
sales_data = pd.read_csv(
    '../Lessons/Lesson 6/Files/SalesData.csv',
    sep=',',
    header=0)
sales_data['OrderDate'] = pd.to_datetime(
    sales_data['OrderDate']
)
total_sales = sales_data[sales_data['OrderDate'].dt.year == 2019].groupby(
        by=sales_data['OrderDate'].dt.month).sum().sort_values(by='Total')
total_sales

/var/folders/g2/jdj1wm7n7lqg0315d1ylkc7xw_5p47/T/ipykernel_42558/1956629609.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  by=sales_data['OrderDate'].dt.month).sum().sort_values(by='Total')


,Units,Unit Cost,Total
OrderDate,,,
3,56,2.99,167.44
8,35,4.99,174.65
4,135,6.98,448.65
9,18,140.99,505.84
5,122,6.98,512.78
2,63,24.98,719.37
11,111,24.98,778.89
10,92,17.98,827.08
6,150,13.98,988.50



&emsp;&emsp; 4. Знайти всіх виконавців, у яких понад 10 альбомів (`chinook/albums`)

In [223]:
import sqlite3  # імпорт модуля для створення з'єднання з БД

# Для створення з'єднання з SQLite БД необхідно передати шлях до файлу, що містить дані
connection = sqlite3.connect('../Lessons/Lesson 6/Files/chinook.db')

albums = pd.read_sql_query(
    sql="SELECT ArtistId, AlbumId FROM albums WHERE AlbumId > 10", 
    con=connection 
)
albums

,ArtistId,AlbumId
0,8,11
1,9,12
2,10,13
3,11,14
4,11,15
...,...,...
332,226,343
333,272,344
334,273,345
335,274,346



&emsp;&emsp; 5. Знайти всіх родичів по прізвищу. В результаті повинна вийти таблиця з двома
колонками. У першій колонці повне ім&#39;я пасажира, у другій його родичів на борту
(вважати родичами пасажирів, у яких збігається прізвище) (Titanic)

In [291]:
from pandas import unique


def find_surname(name):
    last_name = name.split(',')[0]
    return f'{last_name}'

titanic_data_relatives = pd.DataFrame(titanic_data['Name'])
titanic_data_relatives['LastName'] = titanic_data_relatives['Name'].apply(find_surname)
titanic_data_relatives['Relatives'] = np.where(titanic_data_relatives['LastName'] is unique, 'No Relatives', [x + x for x in titanic_data_relatives['Name'].values if any(titanic_data_relatives['LastName'].duplicated())])
titanic_data_relatives.iloc[279]

Name                          Abbott, Mrs. Stanton (Rosa Hunt)
LastName                                                Abbott
Relatives    Abbott, Mrs. Stanton (Rosa Hunt)Abbott, Mrs. S...
Name: 279, dtype: object